 ### Student Matric Number: 22/0041
 ### Department: Software Engineering (Group B)
 ### Course: Artificial Intelligence and Applications (COSC 423)
 ### Date: 17th November, 2025

**COSC 423 Wk 5-6 Lab Work**

1. **Lab Work 01: Graph BFS**

In [27]:
from collections import deque

# Graph representation
graph = {
    "Lagos": ["Ibadan", "Abeokuta"],
    "Ibadan": ["Ilorin", "Osogbo"],
    "Abeokuta": ["Ondo"],
    "Osogbo": ["Akure"],
    "Ondo": ["Akure"],
    "Ilorin": ["Lokoja"],
    "Akure": [],
    "Lokoja": []
}

def bfs_shortest_path(graph, start, goal):
    queue = deque([[start]])   # queue stores paths
    visited = set()

    while queue:
        path = queue.popleft()
        city = path[-1]

        # Goal check
        if city == goal:
            return path

        # Skip visited cities
        if city in visited:
            continue

        visited.add(city)

        # Explore neighbors
        for neighbor in graph[city]:
            new_path = list(path)
            new_path.append(neighbor)
            queue.append(new_path)

    return None

# Run BFS
path = bfs_shortest_path(graph, "Lagos", "Akure")

# Print results
if path:
    print("Shortest Path:", " → ".join(path))
    print("Number of Steps:", len(path) - 1)
else:
    print("No path found!")

Shortest Path: Lagos → Ibadan → Osogbo → Akure
Number of Steps: 3


2. **Lab Work 02: Grid BFS**

**Robot Movement in a Grid**

In [29]:
from collections import deque

# Grid representation (1 = blocked, 0 = free)
grid = [
    [0, 0, 0],
    [0, 1, 0],  # (1,1) is blocked
    [0, 0, 0]
]

# Allowed moves: up, down, left, right
moves = [(0,1), (1,0), (0,-1), (-1,0)]

def bfs(start, goal):
    queue = deque([[start]])  # queue of paths
    visited = set([start])

    while queue:
        path = queue.popleft()
        x, y = path[-1]

        # Goal reached
        if (x, y) == goal:
            return path

        # Expand neighbors
        for dx, dy in moves:
            nx, ny = x + dx, y + dy

            # Check boundaries & blocked cell
            if 0 <= nx < 3 and 0 <= ny < 3 and grid[nx][ny] == 0:
                if (nx, ny) not in visited:
                    visited.add((nx, ny))
                    queue.append(path + [(nx, ny)])

# Run BFS
start = (0, 0)
goal = (2, 2)
shortest_path = bfs(start, goal)

# Output
print("Shortest Path:", shortest_path)
print("Steps:", len(shortest_path) - 1)

Shortest Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]
Steps: 4


3. **Lab Work 03: Word BFS**

In [31]:
from collections import deque


def one_letter_diff(word1, word2):
    """Check if two words differ by exactly one letter"""
    count = 0
    for a, b in zip(word1, word2):
        if a != b:
            count += 1
        if count > 1:
            return False
    return count == 1

def bfs_word_ladder(start, goal, word_list):
    word_list = set(word_list)  # fast lookup
    queue = deque([[start]])
    visited = set([start])

    while queue:
        path = queue.popleft()
        word = path[-1]

        if word == goal:
            return path

        # Check neighbors
        for w in word_list:
            if w not in visited and one_letter_diff(word, w):
                visited.add(w)
                queue.append(path + [w])
    return None

# Example
start = "hit"
goal = "cog"
dictionary = ["hot", "dot", "dog", "lot", "log", "cog"]

path = bfs_word_ladder(start, goal, dictionary)

if path:
    print("Shortest Transformation Path:", path)
    print("Length:", len(path))
else:
    print("No path found!")


Shortest Transformation Path: ['hit', 'hot', 'dot', 'dog', 'cog']
Length: 5


**COSC 423 Wk 7-8 Lab Work**

1. **Lab Work 04: Implementation of the 8-puzzle game**

Goal Test Function and Transition Function

In [15]:
def is_goal(state, goal):
    return state == goal

def neighbors(state):
    # yields (new_state, move_description)
    s = list(state)
    i = s.index(0)
    r, c = divmod(i, 3)
    moves = []
    def swap_and_make(newi, action):
        ns = s.copy()
        ns[i], ns[newi] = ns[newi], ns[i]
        return (tuple(ns), action)

    if r > 0:  # up
        newi = (r-1)*3 + c
        moves.append(swap_and_make(newi, f'Move {s[newi]} down'))
    if r < 2:  # down
        newi = (r+1)*3 + c
        moves.append(swap_and_make(newi, f'Move {s[newi]} up'))
    if c > 0:  # left
        newi = r*3 + (c-1)
        moves.append(swap_and_make(newi, f'Move {s[newi]} right'))
    if c < 2:  # right
        newi = r*3 + (c+1)
        moves.append(swap_and_make(newi, f'Move {s[newi]} left'))

    return moves

Search Algorithms for the 8-puzzle solver.

In [16]:
from collections import deque

def bfs(start, goal):
    # start, goal: tuples of length 9
    if start == goal:
        return [start], []

    frontier = deque([start])
    parent = {start: None}
    move_from_parent = {start: None}

    while frontier:
        state = frontier.popleft()
        if is_goal(state, goal):
            # reconstruct path
            path = []
            moves = []
            cur = state
            while cur is not None:
                path.append(cur)
                moves.append(move_from_parent[cur])
                cur = parent[cur]
            path.reverse()
            moves.reverse()
            # first move is None (start) - chop it for moves list
            return path, [m for m in moves[1:]]
        for ns, mv in neighbors(state):
            if ns not in parent:
                parent[ns] = state
                move_from_parent[ns] = mv
                frontier.append(ns)
    return None

def dfs(start, goal, max_depth=1000):
    stack = [(start, 0)]
    parent = {start : None}
    move_from_parent = {start : None}

    while stack:
        state, depth = stack.pop()
        if is_goal(state, goal):
            path, moves = [], []
            cur = state

            while cur is not None:
                path.append(cur)
                moves.append(move_from_parent[cur])
                cur = parent[cur]
            return path[::-1], [m for m in moves[::-1][1:]]
        
        if depth < max_depth:
            for ns, mv in neighbors(state):
                if ns not in parent:
                    parent[ns] = state
                    move_from_parent[ns] = mv
                    stack.append((ns, depth + 1))
    return None


def dls(start, goal, limit):
    def recursive_dls(state, goal, limit, parent, move_from_parent):
        if is_goal(state, goal):
            return [state], [move_from_parent[state]]
        elif limit == 0:
            return "cutoff"
        else:
            cutoff_occured = False
            for ns, mv in neighbors(state):
                if ns not in parent:
                    parent[ns] = state
                    move_from_parent[ns] = mv
                    result = recursive_dls(ns, goal, limit - 1, parent, move_from_parent)
                    if result == "cutoff":
                        cutoff_occured = True
                    elif result is not None:
                        path, moves = result
                        path.insert(0, state)
                        moves.insert(0, mv)
                        return path, moves
            return "cutoff" if cutoff_occured else None
        
    parent = {start : None}
    move_from_parent = {start : None}
    result = recursive_dls(start, goal, limit, parent, move_from_parent)

    if isinstance(result, tuple):
        path, moves = result
        return path, [m for m in moves[1:]] if len(moves) > 1 else []
    
    return None

def ids(start, goal, max_depth=50):
    for depth in range(max_depth):
        result = dls(start, goal, depth)
        if result is not None:
            return result
    return None

def bidirectional_search(start, goal):
    if start == goal:
        return [start], []

    frontier_start = deque([start])
    frontier_goal = deque([goal])

    parent_start = {start: None}
    parent_goal = {goal: None}
    
    move_from_start = {start: None}
    move_from_goal = {goal: None}

    while frontier_start and frontier_goal:
        # Expand forward frontier
        state_fwd = frontier_start.popleft()
        for ns, mv in neighbors(state_fwd):
            if ns not in parent_start:
                parent_start[ns] = state_fwd
                move_from_start[ns] = mv
                frontier_start.append(ns)
                if ns in parent_goal:
                    # Meeting point found
                    return reconstruct_bidirectional_path(ns, parent_start, parent_goal, move_from_start, move_from_goal)

        # Expand backward frontier
        state_bwd = frontier_goal.popleft()
        for ns, mv in neighbors(state_bwd):
            if ns not in parent_goal:
                parent_goal[ns] = state_bwd
                move_from_goal[ns] = mv
                frontier_goal.append(ns)
                if ns in parent_start:
                    return reconstruct_bidirectional_path(ns, parent_start, parent_goal, move_from_start, move_from_goal)

    return None


def reconstruct_bidirectional_path(meeting_state, parent_start, parent_goal, move_from_start, move_from_goal):
    # Reconstruct forward path
    path_start, moves_start = [], []
    cur = meeting_state
    while cur is not None:
        path_start.append(cur)
        moves_start.append(move_from_start[cur])
        cur = parent_start[cur]
    path_start.reverse()
    moves_start.reverse()

    # Reconstruct backward path
    path_goal, moves_goal = [], []
    cur = meeting_state
    while cur is not None:
        path_goal.append(cur)
        moves_goal.append(move_from_goal[cur])
        cur = parent_goal[cur]

    # Combine both halves (excluding duplicate meeting point)
    full_path = path_start + path_goal[1:]
    full_moves = [m for m in moves_start[1:] + moves_goal[:-1] if m]

    return full_path, full_moves


- Question 1: Algorithm Behavior Comparison

Record the number of steps (path length), number of nodes expanded, and time taken by each algorithm.

In [17]:
import time

def run_experiment(search_fn, start, goal):
    start_time = time.time()
    
    # Track expansions
    global_expanded = 0

    # Wrap neighbors to count expansions
    original_neighbors = neighbors
    def counted_neighbors(state):
        nonlocal global_expanded
        global_expanded += 1
        return original_neighbors(state)

    # Temporarily override neighbors
    globals()['neighbors'] = counted_neighbors

    result = search_fn(start, goal)

    # Restore neighbors
    globals()['neighbors'] = original_neighbors

    end_time = time.time()

    if result is None:
        return None

    path, moves = result
    return {
        "path_length": len(path) - 1,
        "nodes_expanded": global_expanded,
        "time_taken": round(end_time - start_time, 6)
    }

Running the experiment

In [18]:
start_state = (1,2,3,4,0,6,7,5,8)   # example
goal_state  = (1,2,3,4,5,6,7,8,0)

results = {
    "BFS": run_experiment(lambda s,g: bfs(s,g), start_state, goal_state),
    "DFS": run_experiment(lambda s,g: dfs(s,g), start_state, goal_state),
    "DLS": run_experiment(lambda s,g: dls(s,g, limit=20), start_state, goal_state),
    "IDS": run_experiment(lambda s,g: ids(s,g), start_state, goal_state),
    "BIDIRECTIONAL": run_experiment(lambda s,g: bidirectional_search(s,g), start_state, goal_state)
}

for algo, data in results.items():
    print(algo, data)

import pandas as pd
df = pd.DataFrame(results).T  
df


BFS {'path_length': 2, 'nodes_expanded': 8, 'time_taken': 5.4e-05}
DFS {'path_length': 992, 'nodes_expanded': 28685, 'time_taken': 0.137775}
DLS {'path_length': 2, 'nodes_expanded': 3800, 'time_taken': 0.013691}
IDS {'path_length': 2, 'nodes_expanded': 4, 'time_taken': 3e-05}
BIDIRECTIONAL {'path_length': 2, 'nodes_expanded': 2, 'time_taken': 2.2e-05}


,path_length,nodes_expanded,time_taken
BFS,2.0,8.0,0.000054
DFS,992.0,28685.0,0.137775
DLS,2.0,3800.0,0.013691
IDS,2.0,4.0,0.000030
BIDIRECTIONAL,2.0,2.0,0.000022


- Question 2: Optimality and Completeness
  
Modify the start state so that the puzzle is more difficult to solve, for example:

```
start_state = (1, 3, 4, 8, 6, 2, 7, 0, 5)
```

Then re-run all algorithms.

In [19]:
start_state = (1,3,4,8,6,2,7,0,5)
goal_state  = (1,2,3,4,5,6,7,8,0)

results = {
    "BFS": run_experiment(lambda s,g: bfs(s,g), start_state, goal_state),
    "DFS": run_experiment(lambda s,g: dfs(s,g), start_state, goal_state),
    "DLS": run_experiment(lambda s,g: dls(s,g, limit=20), start_state, goal_state),
    "IDS": run_experiment(lambda s,g: ids(s,g), start_state, goal_state),
    "BIDIRECTIONAL": run_experiment(lambda s,g: bidirectional_search(s,g), start_state, goal_state)
}

for algo, data in results.items():
    print(algo, data)


BFS None
DFS None
DLS None
IDS None
BIDIRECTIONAL None


*None of the search algorithms was able to find a solution for the problem.*

- Question 3:Depth Limit Experiment(for DLS)

 Experiment with different depth limits (e.g., 5, 10, 15, 20) for the Depth-Limited Search algorithm.

In [20]:
import time

def dls_experiment(start, goal, limit):
    nodes_expanded = 0
    
    def recursive_dls(state, goal, limit, parent, move_from_parent):
        nonlocal nodes_expanded
        nodes_expanded += 1
        
        if is_goal(state, goal):
            return [state], [move_from_parent[state]]
        
        if limit == 0:
            return "cutoff"
        
        cutoff_occurred = False
        
        for ns, mv in neighbors(state):
            if ns not in parent:
                parent[ns] = state
                move_from_parent[ns] = mv
                result = recursive_dls(ns, goal, limit - 1, parent, move_from_parent)
                
                if result == "cutoff":
                    cutoff_occurred = True
                elif result is not None:
                    path, moves = result
                    path.insert(0, state)
                    moves.insert(0, mv)
                    return path, moves
        
        return "cutoff" if cutoff_occurred else None

    parent = {start: None}
    move_from_parent = {start: None}
    
    start_time = time.time()
    result = recursive_dls(start, goal, limit, parent, move_from_parent)
    end_time = time.time()

    if isinstance(result, tuple):
        path, moves = result
        return {
            "limit": limit,
            "found": True,
            "path_length": len(path),
            "nodes_expanded": nodes_expanded,
            "time": end_time - start_time
        }
    else:
        return {
            "limit": limit,
            "found": False,
            "path_length": None,
            "nodes_expanded": nodes_expanded,
            "time": end_time - start_time
        }

Run the experiment for several depth limits

In [21]:
limits = [10, 20, 50, 100]

results = {}

for L in limits:
    results[f"DLS_limit_{L}"] = dls_experiment(start_state, goal_state, L)

df_dls = pd.DataFrame(results).T
df_dls

,limit,found,path_length,nodes_expanded,time
DLS_limit_10,10,False,None,311,0.000585
DLS_limit_20,20,False,None,10109,0.025236
DLS_limit_50,50,False,None,116770,0.422875
DLS_limit_100,100,False,None,144067,0.674252


- Question 4:Visualize and Explain Search Frontier

  Print or visualize the frontier (open list) and visited nodes (closed list) for BFS and DFS at each iteration.

In [32]:
from collections import deque
import heapq

# Helper: Pretty-print a puzzle state
def print_state(state):
    for i in range(0, 9, 3):
        print(state[i:i+3])
    print()
    
def bfs_visualize(start, goal):
    frontier = deque([start])
    visited = set()

    parent = {start: None}

    iteration = 0
    
    while frontier:
        print(f"\n=== BFS Iteration {iteration} ===")
        iteration += 1
        
        print("Frontier:")
        for f in list(frontier):
            print_state(f)

        print("Visited:")
        for v in visited:
            print_state(v)

        state = frontier.popleft()

        if state == goal:
            print("Goal found!")
            return
        
        visited.add(state)

        for ns, mv in neighbors(state):
            if ns not in visited and ns not in frontier:
                parent[ns] = state
                frontier.append(ns)


def dfs_visualize(start, goal, max_depth=50):
    stack = [(start, 0)]
    visited = set()
    parent = {start: None}

    iteration = 0

    while stack:
        print(f"\n=== DFS Iteration {iteration} ===")
        iteration += 1

        print("Frontier (Stack):")
        for s, d in stack:
            print_state(s)

        print("Visited:")
        for v in visited:
            print_state(v)

        state, depth = stack.pop()

        if state == goal:
            print("Goal found!")
            return

        visited.add(state)

        if depth < max_depth:
            for ns, mv in neighbors(state):
                if ns not in visited:
                    parent[ns] = state
                    stack.append((ns, depth + 1))


How to use the visualizers

In [33]:
start = (1,2,3,4,5,6,7,8,0)
goal  = (1,2,3,4,5,6,7,0,8)

bfs_visualize(start, goal)
dfs_visualize(start, goal)



=== BFS Iteration 0 ===
Frontier:
(1, 2, 3)
(4, 5, 6)
(7, 8, 0)

Visited:

=== BFS Iteration 1 ===
Frontier:
(1, 2, 3)
(4, 5, 0)
(7, 8, 6)

(1, 2, 3)
(4, 5, 6)
(7, 0, 8)

Visited:
(1, 2, 3)
(4, 5, 6)
(7, 8, 0)


=== BFS Iteration 2 ===
Frontier:
(1, 2, 3)
(4, 5, 6)
(7, 0, 8)

(1, 2, 0)
(4, 5, 3)
(7, 8, 6)

(1, 2, 3)
(4, 0, 5)
(7, 8, 6)

Visited:
(1, 2, 3)
(4, 5, 0)
(7, 8, 6)

(1, 2, 3)
(4, 5, 6)
(7, 8, 0)

Goal found!

=== DFS Iteration 0 ===
Frontier (Stack):
(1, 2, 3)
(4, 5, 6)
(7, 8, 0)

Visited:

=== DFS Iteration 1 ===
Frontier (Stack):
(1, 2, 3)
(4, 5, 0)
(7, 8, 6)

(1, 2, 3)
(4, 5, 6)
(7, 0, 8)

Visited:
(1, 2, 3)
(4, 5, 6)
(7, 8, 0)

Goal found!


- Question 5: Design and Extension Challenge

Extend the code to include Uniform Cost Search (UCS) and A* Search for the same 8-puzzle problem.Use a simple heuristic function such as Misplaced Tiles or Manhattan Distance.


In [24]:
def heuristic_misplaced(state, goal):
    return sum(1 for a,b in zip(state, goal) if a!=0 and a!=b)

def heuristic_manhattan(state, goal):
    # index positions
    def pos(index):
        return (index//3, index%3)
    goal_positions = {val: i for i,val in enumerate(goal)}
    total = 0
    for i, val in enumerate(state):
        if val == 0:
            continue
        gi = goal_positions[val]
        r1,c1 = pos(i)
        r2,c2 = pos(gi)
        total += abs(r1-r2)+abs(c1-c2)
    return total

def a_star(start, goal, heuristic):
    # A* returns path and moves list, or None if no solution
    open_set = []
    heapq.heappush(open_set, (heuristic(start, goal), 0, start))
    came_from = {start: None}
    move_from_parent = {start: None}
    gscore = {start: 0}

    while open_set:
        f, g, current = heapq.heappop(open_set)
        if is_goal(current, goal):
            # reconstruct path
            path = []
            moves = []
            cur = current
            while cur is not None:
                path.append(cur)
                moves.append(move_from_parent[cur])
                cur = came_from[cur]
            path.reverse(); moves.reverse()
            return path, [m for m in moves[1:]]
        for ns, mv in neighbors(current):
            tentative_g = gscore[current] + 1
            if ns not in gscore or tentative_g < gscore[ns]:
                gscore[ns] = tentative_g
                came_from[ns] = current
                move_from_parent[ns] = mv
                fscore = tentative_g + heuristic(ns, goal)
                heapq.heappush(open_set, (fscore, tentative_g, ns))
    return None

def uniform_cost_search(start, goal):
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {start: None}
    move_from_parent = {start: None}
    gscore = {start: 0}

    while open_set:
        cost, current = heapq.heappop(open_set)

        if is_goal(current, goal):
            path, moves = [], []
            cur = current
            while cur is not None:
                path.append(cur)
                moves.append(move_from_parent[cur])
                cur = came_from[cur]
            return path[::-1], [m for m in moves[::-1][1:]]
        
        for ns, mv in neighbors(current):
            tentative_g = gscore[current] + 1
            if ns not in gscore or tentative_g < gscore[ns]:
                gscore[ns] = tentative_g
                came_from[ns] = current
                move_from_parent[ns] = mv
                heapq.heappush(open_set, (tentative_g, ns))
    return None

def greedy_best_first_search(start, goal, heuristic):
    open_set = []
    heapq.heappush(open_set, (heuristic(start, goal), start))
    came_from = {start: None}
    move_from_parent = {start: None}
    visited = set()

    while open_set:
        h, current = heapq.heappop(open_set)

        if current in visited:
            continue
        visited.add(current)

        if is_goal(current, goal):
            path, moves = [], []
            cur = current
            while cur is not None:
                path.append(cur)
                moves.append(move_from_parent[cur])
                cur = came_from[cur]
            return path[::-1], [m for m in moves[::-1][1:]]
        
        for ns, mv in neighbors(current):
            if ns not in visited:
                came_from[ns] = current
                move_from_parent[ns] = mv
                heapq.heappush(open_set, (heuristic(ns, goal), ns))

    return None


Running the Experiment

In [34]:
start_state = (1,2,3,4,0,6,7,5,8) 
goal_state  = (1,2,3,4,5,6,7,8,0)

results = {
    "A*": run_experiment(lambda s,g: a_star(s,g, heuristic_misplaced), start_state, goal_state),
    "UCS": run_experiment(lambda s,g: uniform_cost_search(s,g), start_state, goal_state),
    "GBFS": run_experiment(lambda s,g: greedy_best_first_search(s,g, heuristic_misplaced), start_state, goal_state),
}

for algo, data in results.items():
    print(algo, data)

import pandas as pd
df = pd.DataFrame(results).T  
df



A* {'path_length': 2, 'nodes_expanded': 2, 'time_taken': 0.000309}
UCS {'path_length': 2, 'nodes_expanded': 9, 'time_taken': 0.0003}
GBFS {'path_length': 2, 'nodes_expanded': 2, 'time_taken': 0.00023}


,path_length,nodes_expanded,time_taken
A*,2.0,2.0,0.000309
UCS,2.0,9.0,0.000300
GBFS,2.0,2.0,0.000230
